In [36]:
import requests
import pandas as pd
from random_word import Wordnik
import time
import math

In [24]:
def generate_search_strings(num_items: int, counter: int):    
    batch_size = 10
    num_calls = math.ceil(num_items/batch_size)
    output = []
    wordnik_service = Wordnik()
    for i in range(num_calls):
        try:
            # Return a single random word
            res = wordnik_service.get_random_words(includePartOfSpeech ="noun,verb,adverb",hasDictionaryDef=True, limit=batch_size)
            #TODO: Check if adding a duplicate search term
            [output.append(x) for x in res]
        except Exception as e:
            print(e)
    print('Generated ', int(batch_size * num_calls), ' search terms.' )
    return output

def lexica_search(terms: list, counter: int):
    search_base='https://lexica.art/api/v1/search?q='
    prompts = pd.DataFrame(columns=['search_string','source','prompt'])
    print('Starting counter is: ', counter)
    for i in range(counter,len(terms)):
        #print('Searching term: ' , item)
        query = terms[i]
        query = query.replace(' ', '+')
        try:
            d = requests.get(url=(search_base + query))
            data = d.json()
            obj = data['images']
        except Exception as e:
            print(e)
            print('Writing counter to file: ', counter)
            time.sleep(35)
            f = open('./counter.txt', 'w')
            f.write(str(counter))
            f.close()
            return prompts, counter
        #print('Adding items to db for search term: ', item)
        for item in obj:
            row = [query, item['src'], item['prompt']]
            prompts.loc[item['id']] = row
        counter +=1
        print('Commited prompts for term ', counter, ' out of ', len(terms))
        time.sleep(.5)
    f = open('./counter.txt', 'w')
    f.write(counter)
    f.close()
    return prompts, counter



In [25]:
with open('./1000-most-common.txt') as file:
    lines = file.readlines()
    common = [line.rstrip() for line in lines]
with open('./counter.txt') as file:
    lines = file.readlines()
    counter = int(lines[0])

while counter != (len(common)-1):
    print("Starting the procedure again with counter: ", counter)
    res_common, counter = lexica_search(terms = common, counter = counter)
    filename = './prompts-with-common-' + str(counter) + '.json'
    res_common.to_json(filename, orient='split')


Starting the procedure again with counter:  373
Starting counter is:  373
Commited prompts for term  374  out of  1000
Commited prompts for term  375  out of  1000
Commited prompts for term  376  out of  1000
Commited prompts for term  377  out of  1000
Commited prompts for term  378  out of  1000
Commited prompts for term  379  out of  1000
Commited prompts for term  380  out of  1000
Commited prompts for term  381  out of  1000
Commited prompts for term  382  out of  1000
Commited prompts for term  383  out of  1000
Commited prompts for term  384  out of  1000
Commited prompts for term  385  out of  1000
Commited prompts for term  386  out of  1000
Commited prompts for term  387  out of  1000
Commited prompts for term  388  out of  1000
Commited prompts for term  389  out of  1000
Commited prompts for term  390  out of  1000
Commited prompts for term  391  out of  1000
Commited prompts for term  392  out of  1000
Commited prompts for term  393  out of  1000
Commited prompts for term 

KeyboardInterrupt: 

In [38]:
'''import re
import sys
import os'''

'''dir_list = os.listdir(os.getcwd())
search_term = 'prompts-with-common'
frames = []
for item in dir_list:
    if re.search(search_term,item):
        temp_df = pd.read_json(item, orient='split')
        frames.append(temp_df)
res = pd.concat(frames)'''

#res.to_json('./common-df.json',orient='split')
res = pd.read_json('./common-df.json', orient='split')
master = pd.read_json('./master-prompts.json', orient='split')

full = pd.concat([master,res])
full.shape
full.to_csv('./full-prompts.txt')


In [57]:
#Tokenizing prompts
import spacy
import pandas as pd
full = pd.read_json('./full-prompts.json', orient='split')
nlp = spacy.load('en_core_web_sm')
def tokenize(prompt):
    temp = []
    f = nlp(prompt)
    for ent in f.ents:
        temp.append({'token': ent.text,'char_start': ent.start_char, 'char_end': ent.end_char, 'label': None})
    return temp

full['tokens'] = full['prompt'].apply(tokenize)

print(full.head())

                                     search_string  \
0158539f-d69b-4328-b26c-0d3e22795c1c     provokers   
05fee663-7db8-4119-a455-e760bdefd759     provokers   
24b2541b-3bfa-4d7e-983c-680667e0823a     provokers   
3d62e1aa-faa4-4a41-9871-855451deb310     provokers   
41073b1a-8bc3-417e-88e7-633f4123f404     provokers   

                                                                                 source  \
0158539f-d69b-4328-b26c-0d3e22795c1c  https://lexica-serve-encoded-images.sharif.wor...   
05fee663-7db8-4119-a455-e760bdefd759  https://lexica-serve-encoded-images.sharif.wor...   
24b2541b-3bfa-4d7e-983c-680667e0823a  https://lexica-serve-encoded-images.sharif.wor...   
3d62e1aa-faa4-4a41-9871-855451deb310  https://lexica-serve-encoded-images.sharif.wor...   
41073b1a-8bc3-417e-88e7-633f4123f404  https://lexica-serve-encoded-images.sharif.wor...   

                                                                                 prompt  \
0158539f-d69b-4328-b26c-0d3e22795c1c 

In [60]:
#Adding GT for the prompts based off of labels from label studio
import json
f = open('./gt.json')
gt_file = json.load(f)
print('Example entry: ', gt_file[10])

for item in gt_file:
    #find the row in full df where there's match
    
    #search for annotations that line up w/entities
    #case 1: entity not included in annotation, label OTHER
    #case 2: fuzzy match between annotation and token char and end (1 word), label ARTIST
    #case 3: multi-word annotation mapping back to each token fuzzily, label ARTIST


SyntaxError: incomplete input (4083767323.py, line 8)

In [15]:
print(full.loc['0158539f-d69b-4328-b26c-0d3e22795c1c']['tokens'])

<class 'str'>


In [14]:
'''search_base='https://lexica.art/api/v1/search?q='
query='in the style of'
query = query.replace(' ', '+')
d = requests.get(url=(search_base + query))
data = d.json()
obj = data['images']
images = pd.DataFrame(columns=['source','prompt'])
for item in obj:
    row = [item['src'],item['prompt']]
    images.loc[item['id']] = row

images.head()

wordnik_service = Wordnik()
res = wordnik_service.get_random_words(includePartOfSpeech ="noun,verb,adverb",hasDictionaryDef=True, limit=50)

type(res)'''

,source,prompt
00683fc0-3a1d-4890-b674-1efb1e19fd69,https://lexica-serve-encoded-images.sharif.wor...,"a pair of 9 0 s style designer sunglasses, vap..."
01c23712-fdc5-4ef9-b51f-be9da213991b,https://lexica-serve-encoded-images.sharif.wor...,"world war in the year of 2 0 7 0, futuristic t..."
024f2ef0-1abc-4376-8119-04c746581942,https://lexica-serve-encoded-images.sharif.wor...,goddess made of ice long hair like a waterfall...
02b3e3b5-a2cd-41ee-891c-c81138e5ea1d,https://lexica-serve-encoded-images.sharif.wor...,rainbow road by mc escher
07293427-4b43-488a-ae9c-7e5fbe9ea2e9,https://lexica-serve-encoded-images.sharif.wor...,a beautiful and immaculate futuristic city. vi...


list